<a href="https://colab.research.google.com/github/krishthaparubi/DTSC3020_PythonKrishThapa/blob/main/Assignment5_Ch_10%2611.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment5: CRM Cleanup @ **DalaShop**
*Files (Ch.10), Exceptions (Ch.10), Unit Tests (Ch.11), and Regular Expressions*  
.....

**Total: 3 points**  (Two questions, 1.5 pts each)  

> This assignment is scenario-based and aligned with Python Crash Course Ch.10 (files & exceptions), Ch.11 (unit testing with `unittest`), and Regular Expressions.

## Scenario
You are a data intern at an online retailer called **DalaShop**.  
Sales exported a **raw contacts** file from the CRM. It contains customer names, emails, and phone numbers, but the formatting is messy and some emails are invalid.  
Your tasks:

1. **Clean** the contacts (Files + Exceptions + Regex).  
2. **Write unit tests** to make sure your helper functions work correctly and keep working in the future.

## Data file (given by the company): `contacts_raw.txt`
Use this exact sample data (you may extend it for your own testing, but do **not** change it when submitting).  
Run the next cell once to create the file beside your notebook.

In [ ]:
# Create the provided company dataset file
with open("contacts_raw.txt", "w", encoding="utf-8") as f:
    f.write('Alice Johnson <alice@example.com> , +1 (469) 555-1234\nBob Roberts <bob[at]example.com> , 972-555-777\nSara M. , sara@mail.co , 214 555 8888\n"Mehdi A." <mehdi.ay@example.org> , (469)555-9999\nDelaram <delaram@example.io>, +1-972-777-2121\nNima <NIMA@example.io> , 972.777.2121\nduplicate <Alice@Example.com> , 469 555 1234')
print("Wrote contacts_raw.txt with sample DalaShop data.")

## Q1 (1.5 pts) — CRM cleanup with Files, Exceptions, and Regex
Implement `q1_crm_cleanup.py` to:

1. **Read** `contacts_raw.txt` using `pathlib` and `with`. If the file is missing, **handle** it gracefully with `try/except FileNotFoundError` (print a friendly message; do not crash).
2. **Validate emails** with a simple regex (`r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}"`).  
   - Trim whitespace with `strip()` before checking.  
   - Use **full** matching (not partial).
3. **Normalize phone numbers:** remove all non-digits (e.g., with `re.sub(r"\D", "", raw)`).  
   - If the result has **≥ 10 digits**, keep the **last 10 digits**.  
   - Otherwise, return an **empty string** (`""`).
4. **Filter rows:** keep **only** rows with a valid email.
5. **Deduplicate:** remove duplicates by **email** using **case-insensitive** comparison (e.g., `email.casefold()`). **Keep the first occurrence** and drop later duplicates.
6. **Output CSV:** write to `contacts_clean.csv` with **columns exactly** `name,email,phone` (UTF-8).  
7. **Preserve input order:** the order of rows in `contacts_clean.csv` must match the **first appearance** order from the input file. **Do not sort** the rows.

**Grading rubric (1.5 pts):**
- (0.4) File read/write via `pathlib` + graceful `FileNotFoundError` handling  
- (0.5) Correct email regex validation + filtering  
- (0.4) Phone normalization + case-insensitive de-dup (keep first)  
- (0.2) Clean code, clear names, minimal docstrings/comments

In [1]:
import re
import csv
from pathlib import Path

def is_valid_email(email):
    """Check if the email is valid."""
    email = email.strip()
    pattern = r"^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$"
    return re.fullmatch(pattern, email) is not None

def normalize_phone(phone):
    """Remove all non-digits and keep last 10 digits if possible."""
    digits = re.sub(r"\D", "", phone)
    if len(digits) >= 10:
        return digits[-10:]
    return ""

def cleanup_contacts(input_file="contacts_raw.txt", output_file="contacts_clean.csv"):
    """Read contacts, clean data, and save to CSV."""
    try:
        path = Path(input_file)
        lines = path.read_text(encoding="utf-8").splitlines()
    except FileNotFoundError:
        print("File not found. Please make sure contacts_raw.txt exists.")
        return

    clean_contacts = []
    seen_emails = set()

    for line in lines:
        # Split line by comma
        parts = [p.strip() for p in line.split(",")]

        if len(parts) < 2:
            continue

        # Try to get name and email
        name = parts[0].replace('"', '').strip()
        email = ""
        phone = parts[-1].strip()

        # If email is in <...> brackets, extract it
        match = re.search(r"<([^>]+)>", line)
        if match:
            email = match.group(1)
        else:
            # otherwise use the second part (some lines use commas instead)
            email = parts[1].strip()

        # Validate email
        if not is_valid_email(email):
            continue

        # Check for duplicates (case-insensitive)
        email_lower = email.casefold()
        if email_lower in seen_emails:
            continue
        seen_emails.add(email_lower)

        # Clean phone
        phone = normalize_phone(phone)

        clean_contacts.append([name, email, phone])

    # Write to CSV
    with open(output_file, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["name", "email", "phone"])
        writer.writerows(clean_contacts)

    print("contacts_clean.csv created successfully!")

if __name__ == "__main__":
    cleanup_contacts()


File not found. Please make sure contacts_raw.txt exists.


## Q2 (1.5 pts) — Unit testing with `unittest`
Create tests in `test_crm_cleanup.py` that cover at least:

1. **Email validation**: valid/invalid variations.  
2. **Phone normalization**: parentheses, dashes, spaces, country code; too-short cases.  
3. **Parsing**: from a small multi-line string (not from a file), assert the exact structured rows (name/email/phone).  
4. **De-duplication**: demonstrate that a case-variant duplicate email is dropped (first occurrence kept).




In [ ]:
import unittest
from q1_crm_cleanup import is_valid_email, normalize_phone

class TestCRM(unittest.TestCase):

    def test_valid_email(self):
        # Valid
        self.assertTrue(is_valid_email("hello@example.com"))
        self.assertTrue(is_valid_email("john_doe@domain.co"))
        # Invalid
        self.assertFalse(is_valid_email("wrong@domain"))
        self.assertFalse(is_valid_email("name[at]example.com"))

    def test_normalize_phone(self):
        # Normal cases
        self.assertEqual(normalize_phone("(469) 555-1234"), "4695551234")
        self.assertEqual(normalize_phone("+1-972-777-2121"), "9727772121")
        self.assertEqual(normalize_phone("214 555 8888"), "2145558888")
        # Too short
        self.assertEqual(normalize_phone("55555"), "")

if __name__ == "__main__":
    unittest.main()


## Grading rubric (total 3 pts)
- **Q1 (1.5 pts)**  
  - (0.4) File I/O with `pathlib` + graceful `FileNotFoundError` handling  
  - (0.5) Email validation (regex + strip + full match) and filtering  
  - (0.4) Phone normalization and **case-insensitive** de-duplication (keep first)  
  - (0.2) Code clarity (names, minimal docstrings/comments)
- **Q2 (1.5 pts)**  
  - (0.6) Meaningful coverage for email/phone functions (valid & invalid)  
  - (0.6) Parsing & de-dup tests that assert exact expected rows  
  - (0.3) Standard `unittest` structure and readable test names
